# Sparkによる機械学習デモ
## サポートベクタマシンを使った分類を実行します


元ネタ：http://next.rikunabi.com/tech/docs/ct_s03600.jsp?p=002315<br>
第1問：「金貨の真贋を見分けよう」<br>
与えられたデータ（コイン100枚分）を教師とする、クラス分類（教師あり学習）問題です。<br>
教師あり学習（Supervised Learning）とは、分類済みのデータを訓練データとして用いることができる状況で、<br>
新たに与えられたデータに対し分類や回帰を行う手法です。


In [1]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

import numpy as np
import brunel

In [2]:
if os.path.isfile("CodeIQ_mycoins.txt") != True:
    !wget http://dl.dropbox.com/u/110505645/CodeIQ/20121109/CodeIQ_mycoins.zip
    !unzip -j  CodeIQ_mycoins.zip
    !rm -rf CodeIQ_mycoins.zip 
    print "Complete Unzip -> CodeIQ_mycoins.txt."
else:
    print "CodeIQ_mycoins.txt exists."

if os.path.isfile("CodeIQ_auth.txt") != True:
    !wget http://dl.dropbox.com/u/110505645/CodeIQ/20121109/CodeIQ_auth.zip
    !unzip -j  CodeIQ_auth.zip
    !rm -rf CodeIQ_auth.zip 
    print  "CodeIQ_auth.txt exists."
else:
    print "CodeIQ_eaten.txt exists."

CodeIQ_mycoins.txt exists.
CodeIQ_eaten.txt exists.


<h2>正しいデータ（教師データ）をグラフ化します</h2>

In [3]:
codeauth = sc.textFile("/gpfs/global_fs01/sym_shared/YPProdSpark/user/s7d0-33d8c3dc5a198f-0f44a58531ac/notebook/work/CodeIQ_auth.txt").map(lambda line: line.split(" "))
print codeauth.first()

[u'0.745', u'14.385', u'1']


In [9]:
codeauth_df = codeauth.toDF(["volume","weight","auth"]).toPandas()

%brunel data('codeauth_df') x(volume) y(weight) color(auth) 

<IPython.core.display.Javascript object>

<h2>教師データをもとにSVMでモデルを作成します</h2>
本来はデータを分割すべきだが今回は訓練データをそのまま使います。

In [5]:
def makeLabelPoint(line):
    tokens = line
    label = line[2]
    features = line[0:2]
    return LabeledPoint(label, features)

parsedData = codeauth.map(makeLabelPoint)
model = SVMWithSGD.train(parsedData, iterations=1000)

学習データを使ってモデルの精度を確認します。<br>
訓練データを用いて評価を行います。

In [ ]:
labelsAndPreds = parsedData.map(lambda x: (x.label, float(model.predict(x.features))))
trainErr = labelsAndPreds.filter(lambda (x, y): x != y).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

<h2>作成されたモデルをもとに分類を実行しグラフにプロットします</h2>

In [ ]:
fname2 = os.getcwd()  + "/CodeIQ_mycoins.txt"
codecoin = sc.textFile(fname2).map(lambda line: line.split(" "))

type(codecoin)

for i in codecoin.take(codecoin.count()):
    print "モデルによる判定結果：" + str(model.predict([i[0],i[1]]))

In [ ]:
codecoin_df = codecoin.map(lambda x: (x[0],x[1],model.predict([x[0],x[1]]))).toDF(["volume","weight","auth"]).toPandas()

%brunel data('codecoin_df') x(volume) y(weight) color(auth) :: width=800, height=300

In [ ]:
codeauth_df = codeauth.toDF(["volume","weight","auth"]).toPandas()
%brunel data('codeauth_df') x(volume) y(weight) color(auth) :: width=800, height=300

<h2>モデルをセーブします</h2>

In [ ]:
## Save and load model
#model.save(sc, "myModelPath")
#sameModel = SVMModel.load(sc, "myModelPath")
